In [1]:
from OptTensor import OptTensor
from BinaryCrossEntropyLoss import BinaryCrossEntropyLoss
import numpy as np

In [2]:
from torch.utils.data import Dataset, random_split, DataLoader
class MyDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __getitem__(self, index):
        sample = {
            "features": self.x[index],
            "label": self.y[index]
        }
        return sample
    def __len__(self):
        return len(self.x)

In [3]:
dim_in = 100
dim_out = [5000, 300, 2]

In [4]:
encodings = {
    'doc2vec': np.load('./data/d2v.npy'),
    'bigram': np.load('./data/pca_bigram.npy'),
    'tfidf': np.load('./data/pca_tfidf.npy'),
    'sent2vec': np.load('./data/s2v.npy'),
    'word2vec': np.load('./data/w2v.npy'),
}
labels = np.load('./data/label.npy')
choices = np.random.choice(np.arange(0, encodings['tfidf'].shape[0]), size=5000, replace=False)

In [5]:
mydata = MyDataset(encodings['word2vec'][choices], labels[choices])
loader = DataLoader(mydata, batch_size=1)
linear1 = OptTensor(np.random.randn(100, 200))
linear2 = OptTensor(np.random.randn(200, 30))
linear3 = OptTensor(np.random.randn(30, 2))
loss = BinaryCrossEntropyLoss()

In [6]:
for data in loader:
    x = data['features'].numpy()
    y = data['label'].numpy()
    lr1 = linear1(x).relu()
    lr2 = linear2(lr1).relu()
    lr3 = linear3(lr2).relu()
    probs = lr3.softmax(temperature=6)
    current_loss = loss(probs, y)
    current_loss.backward()
    break

OptTensor((), backward_func=BinaryCrossEntropyLoss)
OptTensor((1, 2), backward_func=Softmax)
OptTensor((1, 2), backward_func=ReLU)
OptTensor((1, 2), backward_func=MatMul)
OptTensor((30, 2), backward_func=None)


In [13]:
linear1.grad, 

array([[ 0.        ,  0.        ,  0.        , ...,  0.40139982,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ..., -0.06641238,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.09773739,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ..., -0.3243181 ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.08055662,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ..., -0.2727769 ,
         0.        ,  0.        ]])

In [14]:
linear2.grad

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [12]:
lr3.grad

array([[9988.14349318, 9988.14230777]])